# Integrator Testbed

Experiment with the core integrator across all supported simulation types. Toggle the legacy
comparison when needed, tune particle parameters, and explore output artefacts for debugging or validation.

In [1]:
from __future__ import annotations

import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
while PROJECT_ROOT.name and PROJECT_ROOT.name != "LW_windows":
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

VALIDATION_ROOT = PROJECT_ROOT / "examples" / "validation"
if str(VALIDATION_ROOT) not in sys.path:
    sys.path.insert(0, str(VALIDATION_ROOT))

from testbed_ui import IntegratorTestbedApp

In [ ]:
# Build the reusable widget app and display it in the notebook
app = IntegratorTestbedApp(project_root=PROJECT_ROOT)
app.display()